In [1]:
from langchain_community.document_loaders.recursive_url_loader import RecursiveUrlLoader

In [30]:
from bs4 import BeautifulSoup as Soup

url = "https://docs.python.org/3.9/"
loader = RecursiveUrlLoader(
    url=url, max_depth=6, extractor=lambda x: Soup(x, "html.parser").text
)
result = loader.load()

/usr/lib/python3.11/html/parser.py:170: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  k = self.parse_starttag(i)


In [31]:
from langchain.vectorstores import FAISS
from langchain_google_genai import  GoogleGenerativeAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [32]:
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001",google_api_key="api")

In [33]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(result)

In [34]:
db = FAISS.from_documents(docs, embeddings)
db.save_local("faiss_index")

In [35]:
db = FAISS.load_local("faiss_index",embeddings=embeddings,allow_dangerous_deserialization=True)

In [36]:
from langchain.chains.question_answering import load_qa_chain
from langchain import PromptTemplate

In [37]:
prompt_template = """
  Explore the Text Documents. 
  If the answer is present in the context, provide comprehensive details. 
  If not, derive the answer from the video content anyhow.
  \n\n
  Transcript:\n {context}?\n
  Question: \n{question}\n

  Answer:
"""

prompt = PromptTemplate(template = prompt_template, input_variables = ["context", "question"])

In [53]:
question = "features of python?"

In [54]:
docs = db.similarity_search(question,k=5)
docs

[Document(page_content='What is Python?Â¶\nPython is an interpreted, interactive, object-oriented programming language.  It\nincorporates modules, exceptions, dynamic typing, very high level dynamic data\ntypes, and classes.  It supports multiple programming paradigms beyond\nobject-oriented programming, such as procedural and functional programming.\nPython combines remarkable power with very clear syntax. It has interfaces to\nmany system calls and libraries, as well as to various window systems, and is\nextensible in C or C++.  It is also usable as an extension language for\napplications that need a programmable interface. Finally, Python is portable:\nit runs on many Unix variants including Linux and macOS, and on Windows.\nTo find out more, start with The Python Tutorial.  The Beginnerâ\x80\x99s Guide to\nPython links to other\nintroductory tutorials and resources for learning Python.', metadata={'source': 'https://docs.python.org/3.9/faq/general.html', 'title': 'General Python FA

In [55]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chains.question_answering import load_qa_chain
model = ChatGoogleGenerativeAI(model="gemini-pro",
                             temperature=0.3,google_api_key="api")

In [56]:
chain = load_qa_chain(model, chain_type="stuff", prompt=prompt)

In [57]:
response = chain(
    {"input_documents":docs, "question": question}
    , return_only_outputs=True)
response

{'output_text': 'Python is an interpreted, interactive, object-oriented programming language. It incorporates modules, exceptions, dynamic typing, very high level dynamic data types, and classes. It supports multiple programming paradigms beyond object-oriented programming, such as procedural and functional programming.'}